<section id="back"> </section>

<nav style="margin-top: 20px; padding: 10px; background-color: #f0f0f0; border: 10px solid #ccc;">
    <div style="background-color:#2C41FF">
  <h2 style="margin: 0;text-align: center;">Table of Contents</h2>
    </div>
  <ul style="list-style: none; padding: 0;">
    <li style="margin: 5px 0;"><a href="#preSec" style="text-decoration: none; color: #333;">Dataset Explanation </a></li>
    <li style="margin: 5px 0;"><a href="#sec1" style="text-decoration: none; color: #333;">Imports</a></li>
    <li style="margin: 5px 0;"><a href="#sec2" style="text-decoration: none; color: #333;">EDA</a></li>
    <li style="margin: 5px 0;"><a href="#sec3" style="text-decoration: none; color: #333;">Preprocessing</a></li>
    <li style="margin: 5px 0;"><a href="#sec4" style="text-decoration: none; color: #333;">Modeling</a></li>
    <li style="margin: 5px 0;"><a href="#sec5" style="text-decoration: none; color: #333;">Predictions</a></li>
  </ul>
</nav>

<section id="preSec"> </section>

<!DOCTYPE html>
<html>
<head>
</head>
<body>
    <h1 align="center">Dataset Features</h1>
<table>
  <tr>
    <th>Feature Name</th>
    <th>Explanation</th>
  </tr>
  <tr>
    <td>Demographics</td>
    <td>Information about age and sex of participants.</td>
  </tr>
  <tr>
    <td>Internet Use</td>
    <td>Number of hours of using computer/internet per day.</td>
  </tr>
  <tr>
    <td>Children's Global Assessment Scale</td>
    <td>Numeric scale used by mental health clinicians to rate the general functioning of youths under the age of 18.</td>
  </tr>
  <tr>
    <td>Physical Measures</td>
    <td>Collection of blood pressure, heart rate, height, weight and waist, and hip measurements.</td>
  </tr>
  <tr>
    <td>FitnessGram Vitals and Treadmill</td>
    <td>Measurements of cardiovascular fitness assessed using the NHANES treadmill protocol.</td>
  </tr>
  <tr>
    <td>FitnessGram Child</td>
    <td>Health related physical fitness assessment measuring five different parameters including aerobic capacity, muscular strength, muscular endurance, flexibility, and body composition.</td>
  </tr>
  <tr>
    <td>Bio-electric Impedance Analysis</td>
    <td>Measure of key body composition elements, including BMI, fat, muscle, and water content.</td>
  </tr>
  <tr>
    <td>Physical Activity Questionnaire</td>
    <td>Information about children's participation in vigorous activities over the last 7 days.</td>
  </tr>
  <tr>
    <td>Sleep Disturbance Scale</td>
    <td>Scale to categorize sleep disorders in children.</td>
  </tr>
  <tr>
    <td>Actigraphy</td>
    <td>Objective measure of ecological physical activity through a research-grade biotracker.</td>
  </tr>
  <tr>
    <td>Parent-Child Internet Addiction Test </td>
    <td>20-item scale that measures characteristics and behaviors associated with compulsive use of the Internet including compulsivity, escapism, and dependency.</td>
  </tr>
</table>
    <br>
    <p><b>Special Note - <br></b>
    Note in particular the field PCIAT-PCIAT_Total. The   target sii for this competition is derived from this field as described in the data dictionary: 0 for None, 1 for Mild, 2 for Moderate, and 3 for Severe. Additionally, each participant has been assigned a unique identifier id.</p>
</body>
</html>

<section id="sec1"> </section>
<h1> Imports </h1>

<a href="#back" style="text-decoration: none; color: #333;">Back to table of contents</a>

In [76]:
!pip install shap -- quiet
# Basic imports
import numpy as np
import pandas as pd
import time
import random
import datetime
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="Parameters: { 'verbose' } are not used.")

##################### Preprocessing imports 
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedStratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.inspection import permutation_importance
from sklearn.base import clone

##################### Models
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import SplineTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

##################### optuna library import
import optuna
import shap
random_state = 42
n_splits = 5

ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
######## Download the data 
#train=pd.read_csv("train_preprocessed.csv")
#test=pd.read_csv("test_preprocessed.csv")

train=pd.read_csv("train_processed.csv")
test=pd.read_csv("test_processed.csv")
sample = pd.read_csv('sample_submission.csv')


train = train.dropna(subset='sii').reset_index().drop('index',axis=1)
train = train.select_dtypes(include='number')

In [123]:
train

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
0,5,0,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,...,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,3.0,2.0
1,9,0,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,...,14.97400,39.4497,15.4107,27.0552,NaN,2.340,46.0,64.0,0.0,0.0
2,10,1,71.0,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,...,NaN,NaN,NaN,NaN,NaN,2.170,38.0,54.0,2.0,0.0
3,9,0,71.0,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,...,16.77900,58.9338,26.4798,45.9966,NaN,2.451,31.0,45.0,0.0,1.0
4,13,1,50.0,22.279952,59.5,112.2,NaN,60.0,73.0,102.0,...,20.90200,79.6982,35.3804,63.1265,NaN,4.110,40.0,56.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,8,0,NaN,17.139810,52.5,67.2,25.0,60.0,65.0,112.0,...,15.81500,49.3301,20.2645,36.7181,NaN,3.440,41.0,58.0,2.0,0.0
2732,7,1,NaN,13.927006,48.5,46.6,23.0,65.0,75.0,105.0,...,15.14000,42.8185,18.0937,30.0453,NaN,NaN,48.0,67.0,0.0,1.0
2733,13,0,60.0,16.362460,59.5,82.4,NaN,71.0,70.0,104.0,...,17.97970,66.2889,29.7790,52.8320,NaN,3.260,35.0,50.0,1.0,1.0
2734,11,0,68.0,21.441500,60.0,109.8,NaN,79.0,99.0,116.0,...,21.34030,71.3903,28.7792,54.4630,NaN,2.729,56.0,77.0,0.0,1.0


In [97]:
from sklearn.metrics import cohen_kappa_score
from scipy.optimize import minimize
import numpy as np

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [98]:
# Function to evaluate multiple models and find the best one based on optimized QWK
def evaluate_models(models, X, y, test, n_splits=5):
    """
    Evaluate multiple models and return the best model based on optimized QWK.
    
    Parameters:
        models: list of models to evaluate
        X_train: training features
        y_train: training labels
        X_test: testing features
        y_test: testing labels
        n_splits: number of splits for cross-validation

    Returns:
        best_model: the model with the highest QWK score
        model_scores: dictionary of models and their QWK scores
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    model_scores = {}
    best_model = None
    best_qwk = -np.inf  # Initialize the best QWK as negative infinity
    
    for model in models:
        print(f"Evaluating model: {model}")
        
        oof_non_rounded = np.zeros(len(y))
        test_preds = np.zeros((len(test), n_splits))
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Train the model
            model.fit(X_fold_train, y_fold_train)
            
            # Predict on validation set
            oof_non_rounded[val_idx] = model.predict(X_fold_val)
            
            # Predict on test set
            test_preds[:, fold] = model.predict(test)
        
        # Optimize thresholds on the validation predictions
        initial_thresholds = [0.5, 1.5, 2.5]
        result = minimize(evaluate_predictions, x0=initial_thresholds, 
                          args=(y, oof_non_rounded), method='Nelder-Mead')
        
        # Apply the optimized thresholds
        optimized_thresholds = result.x
        final_predictions = threshold_rounder(oof_non_rounded, optimized_thresholds)
        
        # Calculate QWK for the model
        qwk_score = quadratic_weighted_kappa(y, final_predictions)
        model_scores[model] = qwk_score
        
        print(f"Model: {model}, QWK Score: {qwk_score:.4f}, Optimized Thresholds: {optimized_thresholds}")
        
        # Update the best model if this one is better
        if qwk_score > best_qwk:
            best_qwk = qwk_score
            best_model = model
    
    print(f"\nBest Model: {best_model}, Best QWK Score: {best_qwk:.4f}")
    return best_model, model_scores

In [114]:
!pip install colorama
from colorama import Fore, Style


[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [104]:
from IPython.display import clear_output
def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission

In [118]:
Light = LGBMRegressor(verbose = -1)
XGB_Model = XGBRegressor()
CatBoost_Model = CatBoostRegressor(verbose = False)


voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

Submission1 = TrainML(voting_model, test)

Submission1

Training Folds: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]

Mean Train QWK --> 0.9415
Mean Validation QWK ---> 0.3743


----> || Optimized QWK SCORE ::  0.423


,id,sii
0,00008ff9,2
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,0


In [119]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")


train = train.dropna(subset='sii').reset_index().drop('index',axis=1)
train = train.select_dtypes(include='number')
test = test.select_dtypes(include='number')

train_set = set(train.columns)
test_set = set(test.columns)

# find the difference in cols
cols_diff = train_set - test_set
cols_diff.remove('sii')

train = train.drop(list(cols_diff),axis=1)

In [121]:
Submission1.to_csv("submission.csv")

In [120]:
Light = LGBMRegressor(verbose = -1)
XGB_Model = XGBRegressor()
CatBoost_Model = CatBoostRegressor(verbose = False)


voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

Submission2 = TrainML(voting_model, test)

Submission2

Training Folds: 100%|██████████| 5/5 [00:30<00:00,  6.15s/it]

Mean Train QWK --> 0.9557
Mean Validation QWK ---> 0.3534


----> || Optimized QWK SCORE ::  0.416


,id,sii
0,00008ff9,2
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,0


In [64]:
# from sklearn.model_selection import train_test_split
# X = train[train['sii'].notna()]
# y = X['sii']
# X = X.drop('sii',axis=1)
    
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

# xgb = XGBRegressor()
# ct = CatBoostRegressor(verbose = False)
# lightgbm = LGBMRegressor(verbose=-1)

# models= [xgb,ct]
#best_model, model_scores = evaluate_models(models, X_train, y_train, X_test, n_splits=5)